In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf
import numpy as np
import re
from config import db_password
import sys

C:\Users\LocNguyen\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\LocNguyen\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\LocNguyen\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\LocNguyen\anaconda3\envs\PythonData\l

In [2]:
# Import hours dataset, using project_id as index:
hour_df = pd.read_csv("SQL_Database_Export\\tb_full_hours.csv")
hour_df.head()

,hours_amt,hour_id,date,project_id,emp_id,emp_region
0,1.0,711e1ac4-825d-e611-80e7-5065f38b4491,2016-08-12 00:00:00,808df370-775a-e611-80e7-5065f38b4491,58,A
1,1.0,fa07d119-ae78-e611-80e9-5065f38be0e1,2016-09-07 00:00:00,808df370-775a-e611-80e7-5065f38b4491,58,A
2,1.0,993a7494-b278-e611-80e9-5065f38be0e1,2016-09-09 00:00:00,808df370-775a-e611-80e7-5065f38b4491,58,A
3,1.0,13bf44f6-b178-e611-80e9-5065f38be0e1,2016-09-09 00:00:00,808df370-775a-e611-80e7-5065f38b4491,58,A
4,1.0,1300036e-8a7e-e611-80ea-5065f38be0e1,2016-09-12 00:00:00,808df370-775a-e611-80e7-5065f38b4491,58,A


In [3]:
# Import project dataset using project_id as index:
prj_df = pd.read_csv("SQL_Database_Export\\tb_full_project.csv", index_col=1)
prj_df.head()

,end_date,service_line,team,project_manager,overbudget_yes,budget,building_region,nda
project_id,,,,,,,,
c0a1725b-89f9-e811-a966-000d3a37839b,2019-04-15 00:00:00,Commissioning,SB,51,0,21400.0,SF,False
cffdaaf0-a896-e811-a957-000d3a378ca2,2019-04-01 00:00:00,Commissioning,SB,51,0,31865.0,SF,False
b3ed8234-2775-e611-80ea-5065f38b4491,2017-07-31 00:00:00,Commissioning,SB,51,0,36610.0,SF,False
e4344949-aee8-e611-80f9-5065f38b4491,2017-08-31 00:00:00,Commissioning,SB,51,1,8170.0,SF,False
dda37c80-7e6e-e411-8b85-6c3be5a6aa7c,2017-02-28 00:00:00,Commissioning,SB,51,0,20185.0,SF,False


# Preprocessing data:

### Sum number of hours:

In [4]:
# Groupby and get sum hours_amt:
hours_amt = hour_df[['hours_amt','project_id']].groupby(['project_id']).sum()
hours_amt.head()

,hours_amt
project_id,
0034f66b-b497-e511-80f8-c4346bac0260,37.25
00364584-5916-e711-80f8-e0071b66cf61,61.50
003aae49-0f97-e711-8122-e0071b7458a1,83.50
0072b18b-d890-ea11-a811-000d3a36886f,51.25
00766b60-7634-e311-8b1c-78e3b51146cf,2.50


In [5]:
# Rename column:
hours_amt.columns = ['total_hours']
hours_amt.head()

,total_hours
project_id,
0034f66b-b497-e511-80f8-c4346bac0260,37.25
00364584-5916-e711-80f8-e0071b66cf61,61.50
003aae49-0f97-e711-8122-e0071b7458a1,83.50
0072b18b-d890-ea11-a811-000d3a36886f,51.25
00766b60-7634-e311-8b1c-78e3b51146cf,2.50


### Get first and last date of hour entries:

In [6]:
# Groupby and get first_date:
first_date = hour_df[['date','project_id']].groupby(['project_id']).min()
# Convert to datetime:
first_date['date'] = pd.to_datetime(first_date['date'], infer_datetime_format=True)
# Rename column:
first_date.columns = ['first_date']
first_date.head()

,first_date
project_id,
0034f66b-b497-e511-80f8-c4346bac0260,2015-11-23
00364584-5916-e711-80f8-e0071b66cf61,2017-02-15
003aae49-0f97-e711-8122-e0071b7458a1,2017-08-29
0072b18b-d890-ea11-a811-000d3a36886f,2020-05-11
00766b60-7634-e311-8b1c-78e3b51146cf,2013-11-14


In [7]:
# Groupby and get last_date:
last_date = hour_df[['date','project_id']].groupby(['project_id']).max()
# Convert to datetime:
last_date['date'] = pd.to_datetime(last_date['date'], infer_datetime_format=True)
# Rename column:
last_date.columns = ['last_date']
last_date.head()

,last_date
project_id,
0034f66b-b497-e511-80f8-c4346bac0260,2017-12-28
00364584-5916-e711-80f8-e0071b66cf61,2017-06-19
003aae49-0f97-e711-8122-e0071b7458a1,2018-03-30
0072b18b-d890-ea11-a811-000d3a36886f,2020-07-07
00766b60-7634-e311-8b1c-78e3b51146cf,2014-08-27


### Count emp by project:

In [8]:
# extract df to work and reset index:
emp_count = hour_df[['emp_id','project_id']]
emp_count

,emp_id,project_id
0,58,808df370-775a-e611-80e7-5065f38b4491
1,58,808df370-775a-e611-80e7-5065f38b4491
2,58,808df370-775a-e611-80e7-5065f38b4491
3,58,808df370-775a-e611-80e7-5065f38b4491
4,58,808df370-775a-e611-80e7-5065f38b4491
...,...,...
111285,50,b83e0eb9-c688-e511-80f1-c4346bacb92c
111286,50,b83e0eb9-c688-e511-80f1-c4346bacb92c
111287,50,bafa3ea9-ee2b-e511-80dc-c4346bacdbe8
111288,50,8b848ac8-d536-e511-80e2-c4346bacdbe8


In [9]:
# drop duplicates:
emp_count.drop_duplicates()

,emp_id,project_id
0,58,808df370-775a-e611-80e7-5065f38b4491
99,58,18f2c602-cf36-e811-811c-e0071b66cf61
187,58,523b69c3-f057-e511-80e8-c4346bacb92c
191,58,022f8f9a-6aff-43fe-ad5b-c4c1242a356c
192,58,6c3e4f13-23b6-e411-8d3b-6c3be5a8b8b0
...,...,...
110649,37,bd44481b-99e5-e111-8c2b-78e3b51146cf
110669,37,75b22527-7fba-e111-b5df-78e3b511a685
110713,37,a00867fc-78ba-e111-bad6-78e3b508f847
110828,56,4729ac5a-71ba-e111-b5df-78e3b511a685


In [10]:
# Groupby and count emp by project:
emp_count = emp_count.groupby(['project_id']).count()
emp_count

,emp_id
project_id,
0034f66b-b497-e511-80f8-c4346bac0260,19
00364584-5916-e711-80f8-e0071b66cf61,27
003aae49-0f97-e711-8122-e0071b7458a1,22
0072b18b-d890-ea11-a811-000d3a36886f,23
00766b60-7634-e311-8b1c-78e3b51146cf,3
...,...
ff20d98d-8f4c-e811-8122-e0071b66cf61,3
ff67c709-1601-ea11-a811-000d3a36880e,34
ff721a23-0c29-e211-8d52-78e3b51146cf,47


In [11]:
# Rename column:
emp_count.columns = ['emp_count']
emp_count.head()

,emp_count
project_id,
0034f66b-b497-e511-80f8-c4346bac0260,19
00364584-5916-e711-80f8-e0071b66cf61,27
003aae49-0f97-e711-8122-e0071b7458a1,22
0072b18b-d890-ea11-a811-000d3a36886f,23
00766b60-7634-e311-8b1c-78e3b51146cf,3


## Join all data into one df then keep cleaning:

In [12]:
full_df = prj_df.join(hours_amt).join(first_date).join(last_date).join(emp_count)
full_df.head()

,end_date,service_line,team,project_manager,overbudget_yes,budget,building_region,nda,total_hours,first_date,last_date,emp_count
project_id,,,,,,,,,,,,
c0a1725b-89f9-e811-a966-000d3a37839b,2019-04-15 00:00:00,Commissioning,SB,51,0,21400.0,SF,False,114.00,2018-07-31,2019-05-07,38.0
cffdaaf0-a896-e811-a957-000d3a378ca2,2019-04-01 00:00:00,Commissioning,SB,51,0,31865.0,SF,False,162.75,2018-08-02,2019-04-22,108.0
b3ed8234-2775-e611-80ea-5065f38b4491,2017-07-31 00:00:00,Commissioning,SB,51,0,36610.0,SF,False,84.50,2016-09-07,2017-04-05,60.0
e4344949-aee8-e611-80f9-5065f38b4491,2017-08-31 00:00:00,Commissioning,SB,51,1,8170.0,SF,False,54.50,2017-02-02,2017-04-13,35.0
dda37c80-7e6e-e411-8b85-6c3be5a6aa7c,2017-02-28 00:00:00,Commissioning,SB,51,0,20185.0,SF,False,148.00,2014-11-17,2017-02-08,94.0


In [13]:
# Convert dtype:
full_df['end_date'] = pd.to_datetime(full_df['end_date'], infer_datetime_format=True)
full_df.dtypes

end_date           datetime64[ns]
service_line               object
team                       object
project_manager             int64
overbudget_yes              int64
budget                    float64
building_region            object
nda                          bool
total_hours               float64
first_date         datetime64[ns]
last_date          datetime64[ns]
emp_count                 float64
dtype: object

In [14]:
# Adjust end_date:
full_df['adj_end_date'] = [x if x > y else y for x,y in zip( full_df['end_date'], full_df['last_date'] )]
full_df.head()

,end_date,service_line,team,project_manager,overbudget_yes,budget,building_region,nda,total_hours,first_date,last_date,emp_count,adj_end_date
project_id,,,,,,,,,,,,,
c0a1725b-89f9-e811-a966-000d3a37839b,2019-04-15,Commissioning,SB,51,0,21400.0,SF,False,114.00,2018-07-31,2019-05-07,38.0,2019-05-07
cffdaaf0-a896-e811-a957-000d3a378ca2,2019-04-01,Commissioning,SB,51,0,31865.0,SF,False,162.75,2018-08-02,2019-04-22,108.0,2019-04-22
b3ed8234-2775-e611-80ea-5065f38b4491,2017-07-31,Commissioning,SB,51,0,36610.0,SF,False,84.50,2016-09-07,2017-04-05,60.0,2017-07-31
e4344949-aee8-e611-80f9-5065f38b4491,2017-08-31,Commissioning,SB,51,1,8170.0,SF,False,54.50,2017-02-02,2017-04-13,35.0,2017-08-31
dda37c80-7e6e-e411-8b85-6c3be5a6aa7c,2017-02-28,Commissioning,SB,51,0,20185.0,SF,False,148.00,2014-11-17,2017-02-08,94.0,2017-02-28


In [15]:
# Calculate months duration:
full_df['month_duration'] = (full_df.adj_end_date - full_df.first_date)/np.timedelta64(1, 'M')
full_df.head()

,end_date,service_line,team,project_manager,overbudget_yes,budget,building_region,nda,total_hours,first_date,last_date,emp_count,adj_end_date,month_duration
project_id,,,,,,,,,,,,,,
c0a1725b-89f9-e811-a966-000d3a37839b,2019-04-15,Commissioning,SB,51,0,21400.0,SF,False,114.00,2018-07-31,2019-05-07,38.0,2019-05-07,9.199368
cffdaaf0-a896-e811-a957-000d3a378ca2,2019-04-01,Commissioning,SB,51,0,31865.0,SF,False,162.75,2018-08-02,2019-04-22,108.0,2019-04-22,8.640835
b3ed8234-2775-e611-80ea-5065f38b4491,2017-07-31,Commissioning,SB,51,0,36610.0,SF,False,84.50,2016-09-07,2017-04-05,60.0,2017-07-31,10.743547
e4344949-aee8-e611-80f9-5065f38b4491,2017-08-31,Commissioning,SB,51,1,8170.0,SF,False,54.50,2017-02-02,2017-04-13,35.0,2017-08-31,6.899526
dda37c80-7e6e-e411-8b85-6c3be5a6aa7c,2017-02-28,Commissioning,SB,51,0,20185.0,SF,False,148.00,2014-11-17,2017-02-08,94.0,2017-02-28,27.400973


In [16]:
# Remove unused columns:
clean_df = full_df.drop(['end_date','first_date', 'last_date', 'adj_end_date'], axis=1)
clean_df.head()

,service_line,team,project_manager,overbudget_yes,budget,building_region,nda,total_hours,emp_count,month_duration
project_id,,,,,,,,,,
c0a1725b-89f9-e811-a966-000d3a37839b,Commissioning,SB,51,0,21400.0,SF,False,114.00,38.0,9.199368
cffdaaf0-a896-e811-a957-000d3a378ca2,Commissioning,SB,51,0,31865.0,SF,False,162.75,108.0,8.640835
b3ed8234-2775-e611-80ea-5065f38b4491,Commissioning,SB,51,0,36610.0,SF,False,84.50,60.0,10.743547
e4344949-aee8-e611-80f9-5065f38b4491,Commissioning,SB,51,1,8170.0,SF,False,54.50,35.0,6.899526
dda37c80-7e6e-e411-8b85-6c3be5a6aa7c,Commissioning,SB,51,0,20185.0,SF,False,148.00,94.0,27.400973


In [17]:
clean_df.dtypes

service_line        object
team                object
project_manager      int64
overbudget_yes       int64
budget             float64
building_region     object
nda                   bool
total_hours        float64
emp_count          float64
month_duration     float64
dtype: object

In [18]:
#to csv:
clean_df.to_csv('cleaned_input.csv', index=True)